In [57]:
import pandas as pd
import numpy as np

from statsmodels.tsa.arima.model import ARIMA


In [58]:
df = pd.read_csv(
    "../atp_transformed/2000-2024 players_3.csv",
    parse_dates=["tourney_date"]
)

df = df.sort_values(["player_id", "tourney_date"])

# Convert tourney_date to datetime
df['tourney_date_datetime'] = pd.to_datetime(df['tourney_date'])

# Extract year
df['year'] = df['tourney_date_datetime'].dt.year

df = df[(df["year"] >= 2013) & (df["year"] <= 2023)]


C:\Users\jesse\AppData\Local\Temp\ipykernel_16320\3229348571.py:1: DtypeWarning: Columns (18,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [59]:
pid = 104745  # example: Rafael Nadal

p_df = df[df.player_id == pid].sort_values("year")

# 3-year rolling average of player_rank
p_df["rank_3yr_avg"] = p_df["player_rank"].rolling(3, min_periods=3).mean()

print(p_df[["year", "player_rank", "rank_3yr_avg"]])

        year  player_rank  rank_3yr_avg
463325  2013          5.0           NaN
463389  2013          2.0           NaN
463388  2013          2.0      3.000000
463387  2013          2.0      2.000000
463386  2013          2.0      2.000000
...      ...          ...           ...
463867  2022          5.0      4.333333
463906  2023          2.0      3.666667
463904  2023          2.0      3.000000
463905  2023          2.0      2.000000
463907  2023          2.0      2.000000

[583 rows x 3 columns]


In [60]:
MIN_YEARS = 3  # minimum 3 years for rolling average

if len(p_df) < MIN_YEARS:
    raise ValueError("Not enough data to forecast")


In [ ]:
### TBD : meerdere features

#y = p_df["rank_3yr_avg"].dropna()
#X = p_df.loc[y.index, ["player_age", "days_of_experience"]]

#model = ARIMA(y, exog=X, order=(0,1,0), trend="t")
#res = model.fit()


In [61]:
# Target: 3-year rolling rank average
y = p_df["rank_3yr_avg"].dropna()

model = ARIMA(
    y,
    order=(0,1,0),  # random walk with drift
    trend="t"
)

res = model.fit()
print(res.summary())


                               SARIMAX Results                                
Dep. Variable:           rank_3yr_avg   No. Observations:                  581
Model:                 ARIMA(0, 1, 0)   Log Likelihood                -349.214
Date:                Mon, 15 Dec 2025   AIC                            702.428
Time:                        19:39:01   BIC                            711.154
Sample:                             0   HQIC                           705.830
                                - 581                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0017      0.019     -0.093      0.926      -0.038       0.035
sigma2         0.1952      0.005     39.153      0.000       0.185       0.205
Ljung-Box (L1) (Q):                  96.24   Jarque-

c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [62]:
forecast = res.forecast(steps=1)
rank_1y = forecast.iloc[-1]
print("Predicted player rank in 1 year:", rank_1y)


Predicted player rank in 1 year: 1.9982708583022692


c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [63]:
def rank_to_bin(rank):
    if rank <= 10:
        return 1
    elif rank <= 25:
        return 2
    elif rank <= 50:
        return 3
    elif rank <= 100:
        return 4
    elif rank <= 250:
        return 5
    else:
        return 6

predicted_bin = rank_to_bin(rank_1y)
print("Predicted rank bin:", predicted_bin)


Predicted rank bin: 1


In [64]:
forecast_obj = res.get_forecast(steps=1)
conf_int = forecast_obj.conf_int()
conf_int_1y = conf_int.iloc[-1]

print("95% confidence interval for player rank in 1 year:")
print(conf_int_1y)


95% confidence interval for player rank in 1 year:
lower rank_3yr_avg    1.132323
upper rank_3yr_avg    2.864219
Name: 581, dtype: float64


c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
